# Inference
This notebook is intended to depict the results of inference for the xView2 Dataset. 

In [1]:
from utils.inference_step import inference
from utils.helperfunctions import load_checkpoint, find_best_checkpoint, get_data_folder
from utils.dataset import xView2Dataset, collate_fn_test, image_transform
from pathlib import Path
from torch.utils.data import DataLoader
import os
import yaml
from model.siameseNetwork import SiameseUnet
from model.uNet import UNet_ResNet50
from model.loss import FocalLoss, combined_loss_function
import torch
base_dir = Path(os.getcwd()).parent  # Gehe einen Ordner zurück vom aktuellen Arbeitsverzeichnis
config_path = base_dir / "notebooks" / "config1.yaml"
print(base_dir)

with open(config_path, "r") as file:
    config = yaml.safe_load(file)



/dss/dsshome1/08/di97ren/04-geo-oma24/xView2SiameseUNet


## Get all pathes and create Dataset and DataLoader

In [2]:

DATA_ROOT, TEST_ROOT, VAL_IMG, TEST_LABEL, TEST_TARGET, TEST_PNG_IMAGES = get_data_folder("test", main_dataset = False)

test_dataset = xView2Dataset(png_path=TEST_PNG_IMAGES,
image_transform = image_transform(), inference = True)

test_dataloader = DataLoader(
    test_dataset, batch_size = 32,
    collate_fn = collate_fn_test,
    shuffle = False,
    num_workers = 5
)

In [3]:
USER = config["data"]["user"]
#USER_PATH = Path(f"/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/users/{USER}")
USER_HOME_PATH = Path(f"/dss/dsshome1/08/{USER}")

# Pathes to store experiment informations in:
EXPERIMENT_GROUP = config["data"]["experiment_group"]
EXPERIMENT_ID = config["data"]["experiment_id"]
EXPERIMENT_DIR = USER_HOME_PATH / EXPERIMENT_GROUP / "tensorboard_logs" / EXPERIMENT_ID
EXPERIMENT_DIR.mkdir(parents=True, exist_ok=True)

print(EXPERIMENT_DIR)

# Auch Checkpoints-Verzeichnis erstellen
CHECKPOINTS_DIR = USER_HOME_PATH / EXPERIMENT_GROUP / "checkpoints"
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

# Logfiles-Verzeichnis erstellen
LOGFILES_DIR = USER_HOME_PATH / EXPERIMENT_GROUP / "logfiles" / EXPERIMENT_ID
LOGFILES_DIR.mkdir(parents=True, exist_ok=True)
print(f"Logfiles werden gespeichert in: {LOGFILES_DIR}")


/dss/dsshome1/08/di97ren/xView2_all_data/tensorboard_logs/001
Logfiles werden gespeichert in: /dss/dsshome1/08/di97ren/xView2_all_data/logfiles/001


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"


# Modell initialisieren
model = SiameseUnet(num_pre_classes=2, num_post_classes=6)
model.to(device)

best_checkpoint_path = find_best_checkpoint(CHECKPOINTS_DIR, EXPERIMENT_ID)
# Besten Checkpoint laden
model = load_checkpoint(model, best_checkpoint_path)
results = inference(model, test_dataloader)


Loaded raw state_dict from /dss/dsshome1/08/di97ren/xView2_all_data/checkpoints/001_best_siamese_unet_state.pth
Checkpoint erfolgreich in DataParallel-Modell geladen.


Running inference:  68%|██████▊   | 40/59 [00:59<00:22,  1.20s/batch, Samples=1280, Avg time/batch=1.0751s]

: 

: 

# Plot the results:

In [ ]:
from utils.viz import vizualize_predictions

In [ ]:
vizualize_predictions(results)

-------------------------------------------------------

In [1]:

import os
import subprocess
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import time
from IPython.display import display, HTML
import yaml

base_dir = Path(os.getcwd()).parent  # Gehe einen Ordner zurück vom aktuellen Arbeitsverzeichnis
config_path = base_dir / "notebooks" / "config1.yaml"
print(base_dir)

with open(config_path, "r") as file:
    config = yaml.safe_load(file)


/dss/dsshome1/08/di97ren/04-geo-oma24/xView2SiameseUNet


In [2]:
USER = config["data"]["user"]
USER_HOME_PATH = Path(f"/dss/dsshome1/08/{USER}")

# Pathes to store experiment informations in:
EXPERIMENT_GROUP = config["data"]["experiment_group"]
EXPERIMENT_ID = config["data"]["experiment_id"]

USER_HOME_PATH = Path(f"/dss/dsshome1/08/{USER}")
RESULTS_DIR = USER_HOME_PATH / EXPERIMENT_GROUP / "inference_results" / EXPERIMENT_ID
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Inference-Ergebnisse werden gespeichert in: {RESULTS_DIR}")


Inference-Ergebnisse werden gespeichert in: /dss/dsshome1/08/di97ren/xView2_all_data/inference_results/001


In [3]:
existing_results = list(RESULTS_DIR.glob("inference_results_*.pkl"))
if existing_results:
    print("Gefundene bestehende Inference-Ergebnisse:")
    for i, result_file in enumerate(existing_results):
        file_time = time.ctime(result_file.stat().st_mtime)
        file_size = result_file.stat().st_size / (1024 * 1024)  # MB
        print(f"{i+1}. {result_file.name} - {file_time} ({file_size:.2f} MB)")
    
    use_existing = input("\nBestehende Ergebnisse verwenden? (y/n): ").lower()
    if use_existing == 'y':
        idx = int(input("Nummer des zu verwendenden Ergebnisses eingeben: ")) - 1
        results_file = existing_results[idx]
        with open(results_file, 'rb') as f:
            results = pickle.load(f)
        print(f"Ergebnisse aus {results_file} geladen.")
    else:
        # Inference-Skript ausführen
        run_inference = True
else:
    run_inference = True

In [10]:
if 'run_inference' in locals() and run_inference:
    print("Starte Inference-Prozess...")
    try:
        result = subprocess.run([
            "python", "inference_subprocess.py",
            "--user", USER,
            "--experiment_group", config["data"]["experiment_group"],
            "--experiment_id", config["data"]["experiment_id"],
            "--batch_size", "8",
            "--save_results"
        ], capture_output=True, text=True, check=False)  # check=False um die Exception zu vermeiden
        
        if result.returncode != 0:
            print("Fehler beim Ausführen des Inference-Skripts:")
            print(result.stderr)
        else:
            print("Inference erfolgreich abgeschlossen!")
            print(result.stdout)
    except Exception as e:
        print(f"Fehler beim Ausführen des Subprozesses: {e}")
    # try:
    #     # Du kannst hier Argumente anpassen
    #     result = subprocess.run([
    #         "python", "utils/inference_subprocess.py",
    #         "--user", USER,
    #         "--experiment_group", EXPERIMENT_GROUP,
    #         "--experiment_id", EXPERIMENT_ID,
    #         "--batch_size", "8",
    #         "--save_results"
    #     ], capture_output=True, text=True, check=True)
        
    #     print("Inference erfolgreich abgeschlossen!")
    #     print(result.stdout)
        
    #     # Suche nach der neuesten Ergebnis-Datei
    #     latest_result = max(RESULTS_DIR.glob("inference_results_*.pkl"), key=os.path.getmtime)
    #     with open(latest_result, 'rb') as f:
    #         results = pickle.load(f)
    #     print(f"Neueste Ergebnisse aus {latest_result} geladen.")
        
    # except subprocess.CalledProcessError as e:
    #     print("Fehler beim Ausführen des Inference-Skripts:")
    #     print(e.stderr)
    #     raise


Starte Inference-Prozess...
Fehler beim Ausführen des Inference-Skripts:
Traceback (most recent call last):
  File "/dss/dsshome1/08/di97ren/04-geo-oma24/xView2SiameseUNet/notebooks/inference_subprocess.py", line 143, in <module>
    model = load_checkpoint(model, best_checkpoint_path)
  File "/dss/dsshome1/08/di97ren/04-geo-oma24/xView2SiameseUNet/notebooks/utils/inference.py", line 21, in load_checkpoint
    state_dict = checkpoint['model_state_dict']
                 ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^
KeyError: 'model_state_dict'



In [ ]:
# Performance-Metriken anzeigen
if 'results' in locals():
    perf = results['performance']
    print(f"Anzahl der verarbeiteten Samples: {perf['total_samples']}")
    print(f"Gesamte Verarbeitungszeit: {perf['total_processing_time']:.2f} Sekunden")
    print(f"Durchschnittliche Zeit pro Sample: {perf['avg_time_per_sample']:.4f} Sekunden")
    
    print("\nPre-Disaster Klassenverteilung:")
    for cls, count in perf['pre_class_distribution'].items():
        print(f"  Klasse {cls}: {count} Pixel")
    
    print("\nPost-Disaster Klassenverteilung:")
    for cls, count in perf['post_class_distribution'].items():
        print(f"  Klasse {cls}: {count} Pixel")

# %%
# Visualisierungsfunktion importieren und ausführen
from visualize import visualize_predictions  # Passe an deine Importstruktur an

# Visualisiere eine bestimmte Anzahl von Samples
num_samples_to_visualize = 5
save_dir = RESULTS_DIR / "visualizations"
visualize_predictions(results, num_samples=num_samples_to_visualize, save_dir=save_dir)

# Zeige einige der generierten Bilder im Notebook an
visualizations = list(save_dir.glob("prediction_*.png"))
for vis_file in visualizations[:3]:  # Zeige die ersten 3 Bilder
    display(HTML(f'<img src="{vis_file}" width="800">'))

# Zeige auch die Klassenverteilung an
display(HTML(f'<img src="{save_dir}/class_distribution.png" width="800">'))